In [15]:
#Import thư viện cần thiết
from pdf2image import convert_from_path
import os
import re
import unicodedata
from collections import defaultdict
import difflib
import pandas as pd
from PIL import Image
import pytesseract
import cv2
import numpy as np
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR


1. Chuyển file PDF sang các file ảnh

In [4]:
#Xử lí PDF 1 thành hình ảnh dạng png từ trang 203 đến trang 395 (theo id)
pdf_file = "../data/pdf/PDF1.pdf"
output_folder = "../image/PDF1"


images = convert_from_path(
    pdf_file,
    first_page=203,
    last_page=395
)

for page_num, img in enumerate(images, start=203):
    filename = f"PDF1-{page_num}.png"
    save_path = os.path.join(output_folder, filename)
    img.save(save_path, "PNG")


In [5]:
#Xử lí PDF 3 thành hình ảnh dạng png từ trang 12 đến trang 118 (theo id)
pdf_file = "../data/pdf/PDF3.pdf"
output_folder = "../image/PDF3"

os.makedirs(output_folder, exist_ok=True)

start_page = 12
end_page = 118

images = convert_from_path(pdf_file, first_page=start_page, last_page=end_page)

for page_num, img in zip(range(start_page, end_page + 1), images):
    page_str = f"{page_num:03d}"           
    filename = f"PDF3-{page_str}.png"
    save_path = os.path.join(output_folder, filename)

    img.save(save_path, "PNG")

2. Tiền xử lí ảnh trước khi OCR

In [17]:
# =====================================================
INPUT_ROOT = '../image'
INPUT_SUBDIR = 'PDF1'
INPUT_DIR = os.path.join(INPUT_ROOT, INPUT_SUBDIR)

OUTPUT_SUBDIR = 'PDF1_processed'
OUTPUT_DIR = os.path.join(INPUT_ROOT, OUTPUT_SUBDIR)

START_PAGE = 250
END_PAGE = 296

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"✅ Đã tạo thư mục đầu ra: {OUTPUT_DIR}")

# =====================================================
# 2. LỌC FILE THEO SỐ TRANG (GIỮ NGUYÊN)
# =====================================================
target_files = []
pattern = re.compile(r'PDF1-(\d+)\.png$', re.IGNORECASE)

try:
    all_files = os.listdir(INPUT_DIR)
    for file_name in all_files:
        match = pattern.match(file_name)
        if match:
            page_number = int(match.group(1))
            if START_PAGE <= page_number <= END_PAGE:
                target_files.append(file_name)
except FileNotFoundError:
    print(f"❌ Không tìm thấy thư mục đầu vào: {INPUT_DIR}")

target_files.sort(key=lambda f: int(pattern.search(f).group(1)))
print(f"📄 Tìm thấy {len(target_files)} ảnh cần xử lý từ trang {START_PAGE} đến {END_PAGE}")

# =====================================================
# 3. HÀM XỬ LÝ TỐI ƯU OCR: CHỐNG MẤT NÉT & LÀM ĐẬM CHỮ
# =====================================================
def process_optimized_for_ocr(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None, None

    # 1. Chuyển sang Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. Tăng cường độ tương phản cục bộ (CLAHE)
    # Giúp các dòng chữ nhạt màu như "số 641" hiện rõ hơn khỏi nền
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    # 3. Adaptive Thresholding (Phân ngưỡng thích nghi)
    # Thay vì dùng ngưỡng cố định cho cả ảnh, nó tính toán từng vùng nhỏ
    # giúp giữ lại nét chữ mảnh cực tốt.
    binary = cv2.adaptiveThreshold(
        enhanced, 255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 21, 15
    )

    # 4. KHỬ NHIỄU NHẸ (Dùng Median Blur kích thước nhỏ nhất)
    # Để xóa các đốm đen li ti mà không làm hỏng cấu trúc chữ
    denoised = cv2.medianBlur(binary, 3)

    # 5. LÀM ĐẬM NÉT CHỮ (Erosion)
    # Với ảnh nền trắng chữ đen, phép Erode sẽ làm các vùng đen (chữ) dày lên.
    # Điều này cực kỳ quan trọng để Tesseract nhận diện được các chữ bị nhạt/đứt nét.
    kernel = np.ones((2, 2), np.uint8)
    final_img = cv2.erode(denoised, kernel, iterations=1)

    return final_img, img

# =====================================================
# 4. THỰC THI XỬ LÝ ĐỒNG LOẠT
# =====================================================
print(f"🚀 Đang xử lý đồng loạt...")

count = 0
for file_name in target_files:
    full_path = os.path.join(INPUT_DIR, file_name)
    processed_img, _ = process_optimized_for_ocr(full_path)

    if processed_img is not None:
        output_path = os.path.join(OUTPUT_DIR, f"processed_{file_name}")
        cv2.imwrite(output_path, processed_img)
        count += 1
        if count % 10 == 0:
            print(f"✅ Đã xử lý {count}/{len(target_files)} ảnh...")

print(f"\n✨ Hoàn tất! Đã lưu {count} ảnh vào: {OUTPUT_DIR}")

📄 Tìm thấy 47 ảnh cần xử lý từ trang 250 đến 296
🚀 Đang xử lý đồng loạt...
✅ Đã xử lý 10/47 ảnh...
✅ Đã xử lý 20/47 ảnh...
✅ Đã xử lý 30/47 ảnh...
✅ Đã xử lý 40/47 ảnh...

✨ Hoàn tất! Đã lưu 47 ảnh vào: ../image\PDF1_processed


3. Xử lí các ảnh thông qua sử dụng Paddle OCR và Teserract OCR. 

In [21]:
TESSERACT_LANGS = 'vie'

IMG_DIR = "../image/PDF1_processed"

OUT_DIR = "../OCR"

os.makedirs(OUT_DIR, exist_ok=True)



START_PAGE = 250

END_PAGE = 296



# Khởi tạo PaddleOCR

paddle_engine = PaddleOCR(use_angle_cls=True, lang='ch', show_log=False)



# ===============================

# 2. HÀM HỖ TRỢ

# ===============================

def strip_accents(s: str) -> str:

    return "".join(

        c for c in unicodedata.normalize("NFD", s)

        if unicodedata.category(c) != "Mn"

    )



def is_chinese(text: str) -> bool:

    return any('\u4e00' <= ch <= '\u9fff' or '\u3400' <= ch <= '\u4DBF' for ch in text)



def is_page_number_line(text: str) -> bool:

    return re.fullmatch(r"^\s*\d{1,4}\s*$", text) is not None



def looks_like_vietnamese(text: str) -> bool:

    norm = strip_accents(text).lower()

    tokens = re.findall(r"[a-z0-9]+", norm)

    def has_similar(target: str, thr: float) -> bool:

        for t in tokens:

            if difflib.SequenceMatcher(None, t, target).ratio() >= thr: return True

        return False

    return (has_similar("buc", 0.7) and has_similar("thu", 0.7) and

            has_similar("viet", 0.7) and has_similar("cho", 0.7))



def extract_id(text: str):

    def norm_digits(tok: str):

        tok = tok.strip().replace('O', '0').replace('o', '0').replace('I', '1').replace('l', '1').replace('|', '1').replace('!', '1')

        digits = "".join(re.findall(r"\d+", tok))

        if not digits or len(digits) < 1: return None, None

        return digits, None



    m_zh_full = re.search(r"写\s*给\s*自己\s*.*第\s*([0-9OoIl|!\s]{1,20})\s*封\s*信", text)

    if m_zh_full:

        id_val, _ = norm_digits(m_zh_full.group(1))

        if id_val: return id_val, 'ZH_FULL'



    norm = strip_accents(text).lower()

    m_vi_full = re.search(r"buc\s*thu\s*viet\s*cho\s*chinh\s*minh\s*(?:so|s0|s6|s)\s*([0-9OoIl|!\s]{1,20})", norm)

    if m_vi_full:

        id_val, _ = norm_digits(m_vi_full.group(1))

        if id_val: return id_val, 'VI_FULL'



    m_zh_anchor = re.search(r"第\s*([0-9OoIl|!\s]{1,20})", text)

    if m_zh_anchor:

        id_val, _ = norm_digits(m_zh_anchor.group(1))

        if id_val: return id_val, 'ZH_ANCHOR'



    return None, None



# ===============================

# 3. HÀM OCR KẾT HỢP (HYBRID)

# ===============================

def hybrid_ocr(img_path: str):

    img = cv2.imread(img_path)

    results = paddle_engine.ocr(img_path, cls=True)

    final_lines = []

    if not results or results[0] is None: return []



    for line in results[0]:

        box = line[0]

        paddle_text = line[1][0].strip()

        if not is_chinese(paddle_text) and len(paddle_text) > 2:

            try:

                pts = np.array(box, dtype=np.int32)

                x, y, w, h = cv2.boundingRect(pts)

                roi = img[max(0, y-5):y+h+5, max(0, x-5):x+w+5]

                roi_pil = Image.fromarray(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))

                tess_text = pytesseract.image_to_string(roi_pil, lang=TESSERACT_LANGS, config='--psm 7').strip()

                final_lines.append(tess_text if len(tess_text) > 1 else paddle_text)

            except:

                final_lines.append(paddle_text)

        else:

            final_lines.append(paddle_text)

    return final_lines



# =====================================================

# 4. XỬ LÝ DỮ LIỆU (CẬP NHẬT LOGIC CHUYỂN TRANG)

# =====================================================

DATA_MAP = defaultdict(lambda: {"src": [], "tgt": [], "vi_started": False})

STATE = {"current_id": None, "mode": None, "pending_src": []}



def process_image_to_data(img_path: str, data_map, state):

    lines = hybrid_ocr(img_path)

   

    current_id = state["current_id"]

    mode = state["mode"]

    pending_src = state["pending_src"]



    for text in lines:

        if is_page_number_line(text): continue

       

        found_id, id_type = extract_id(text)



        # Nếu tìm thấy ID mới (Tiêu đề bài mới)

        if found_id:

            current_id = found_id

            if id_type in ('ZH_FULL', 'ZH_ANCHOR'):

                mode = "zh"

                if pending_src:

                    data_map[current_id]["src"].extend(pending_src)

                    pending_src.clear()

                data_map[current_id]["src"].append(text)

                data_map[current_id]["vi_started"] = False

            elif id_type in ('VI_FULL', 'VI_ANCHOR'):

                mode = "vi"

                data_map[current_id]["vi_started"] = True

                data_map[current_id]["tgt"].append(text)

            continue



        if current_id is None:

            if is_chinese(text): pending_src.append(text)

            continue



        # --- LOGIC PHÂN LOẠI DÒNG TEXT ---

        has_zh = is_chinese(text)

        is_vi_hint = looks_like_vietnamese(text)



        # 1. Nếu gặp dấu hiệu tiếng Việt (Bức thư viết cho...) -> Chuyển mode VI ngay

        if is_vi_hint:

            mode = "vi"

            data_map[current_id]["vi_started"] = True

            data_map[current_id]["tgt"].append(text)

       

        # 2. Nếu đang ở mode tiếng Việt

        elif mode == "vi":

            # Nếu đang VI mà tự dưng gặp chữ Trung -> Khả năng là ID mới nhưng OCR tiêu đề lỗi

            if has_zh:

                mode = "zh"

                data_map[current_id]["src"].append(text)

            else:

                data_map[current_id]["tgt"].append(text)



        # 3. Nếu đang ở mode tiếng Trung (hoặc Pinyin)

        else: # mode == "zh"

            if has_zh:

                data_map[current_id]["src"].append(text)

            else:

                # Nếu không có chữ Trung: kiểm tra xem đã bắt đầu phần Việt chưa

                if data_map[current_id]["vi_started"]:

                    mode = "vi"

                    data_map[current_id]["tgt"].append(text)

                else:

                    # Vẫn coi là src (Pinyin) nếu chưa có hint Việt rõ ràng

                    data_map[current_id]["src"].append(text)



    # Cập nhật trạng thái sau khi xong 1 ảnh

    state["current_id"] = current_id

    state["mode"] = mode

    state["pending_src"] = pending_src



# =====================================================

# 5. CHẠY VÀ LƯU

# =====================================================

for p in range(START_PAGE, END_PAGE + 1):

    img_path = os.path.join(IMG_DIR, f"processed_PDF1-{p}.png")

    if os.path.exists(img_path):

        print(f"Đang xử lý: {img_path}")

        process_image_to_data(img_path, DATA_MAP, STATE)



rows = []

for sid, v in DATA_MAP.items():

    src = " ".join(v["src"]).strip()

    tgt = " ".join(v["tgt"]).strip()

    if src or tgt:

        rows.append({"src_id": sid, "src_lang": src, "tgt_lang": tgt})



df = pd.DataFrame(rows)

if not df.empty:

    df['src_id_int'] = pd.to_numeric(df['src_id'], errors='coerce')

    df = df.dropna(subset=['src_id_int'])

    df = df[(df['src_id_int'] >= 626) & (df['src_id_int'] <= 750)]

    df = df.sort_values('src_id_int').drop(columns=['src_id_int']).reset_index(drop=True)



    out_csv = os.path.join(OUT_DIR, "PDF1_626_750.csv")

    df.to_csv(out_csv, index=False, encoding="utf-8-sig")

    print(f"Hoàn thành! Đã lưu: {out_csv}")

Đang xử lý: ../image/PDF1_processed\processed_PDF1-250.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-251.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-252.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-253.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-254.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-255.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-256.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-257.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-258.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-259.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-260.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-261.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-262.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-263.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-264.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-265.png
Đang xử lý: ../image/PDF1_processed\processed_PDF1-266.p

4. Hậu xử lí OCR

In [38]:
def clean_src_lang(text):
    if pd.isna(text):
        return text

    # 1. Xóa mọi loại dấu nháy
    text = re.sub(r'["\'`“”‘’‟„«»]', '', text)

    # 2. FIX OCR: xóa ký tự rác đứng ngay trước chữ 写
    text = re.sub(r'[飞以,，]\s*(?=写)', '', text)

    # 3. Cắt từ chữ Hán đầu tiên
    match_start = re.search(r'[\u4e00-\u9fff]', text)
    if match_start:
        text = text[match_start.start():]

    # 4. Cắt phần phiên âm (Pinyin)
    match_pinyin = re.search(r'[a-zA-Zāáǎàēéěèīíǐìōóǒòūúǔùǖǘǚǜ]', text)
    if match_pinyin:
        text = text[:match_pinyin.start()]

    # 5. Xóa ký tự rác trước & sau dấu câu tiếng Trung
    text = re.sub(r'[^。\u4e00-\u9fff]\s*。', '。', text)
    text = re.sub(r'。\s*[^。\u4e00-\u9fff]+', '。', text)

    return text.strip()

def clean_tgt_lang(text):
    if pd.isna(text):
        return text

    # 1. Đồng bộ hóa: Cắt từ cụm "Bức thư"
    marker = "Bức thư"
    idx = text.find(marker)
    if idx != -1:
        text = text[idx:]

    # 2. XÓA TRIỆT ĐỂ DẤU NHÁY (bao gồm cả nháy đơn ' và các loại nháy lạ)
    text = re.sub(r'["\'`“”‘’‟„«»]', '', text)

    # 3. XÓA KÝ TỰ RÁC ASCII
    # Loại bỏ các ký tự như: | ~ * ^ % $ # @ + = < > / \ _
    text = re.sub(r'[|~*^#$%@\+=\[\]{}<>/\\]', '', text)

    # 4. LÀM SẠCH DẤU CÂU TIẾNG VIỆT
    # Xóa khoảng trắng thừa trước dấu chấm, phẩy
    text = re.sub(r'\s+([.,!?])', r'\1', text)
    # Xóa các ký tự không phải chữ/số dính sát vào dấu chấm
    text = re.sub(r'[^.\wÀ-ỹ\d\s](?=\.)', '', text)
    text = re.sub(r'(?<=\.)[^.\wÀ-ỹ\d\s]', '', text)

    # 5. CHUẨN HÓA KHOẢNG TRẮNG
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# ===============================
# QUY TRÌNH XỬ LÝ FILE
# ===============================
input_file = '../OCR/PDF1_626_750.csv'
output_file = '../OCR/PDF1_626_750_cleaned.csv'

try:
    # Đọc từ file gốc
    df = pd.read_csv(input_file)

    # Áp dụng các hàm làm sạch
    df['src_lang'] = df['src_lang'].apply(clean_src_lang)
    df['tgt_lang'] = df['tgt_lang'].apply(clean_tgt_lang)

    # Lưu vào file cleaned
    df.to_csv(output_file, index=False, encoding='utf-8-sig') 
except FileNotFoundError:
    print(f"❌ Không tìm thấy file tại đường dẫn: {input_file}")